In [7]:
from transformers import GPT2LMHeadModel
import torch
from train_gpt2 import GPT, GPTConfig
from torch.nn import functional as F

In [8]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2")
sd_hf = model_hf.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

<!-- from tranformers import pipeline, set_seed
set_seed(42)
generator("Hello, I am a language model") -->

In [9]:
num_return_sequences = 5
max_length = 30

#model = GPT.from_pretrained('gpt2')
model = GPT(GPTConfig())
model.eval()
model.to('cuda')

import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode('Knock Knock, Who is there?')
tokens = torch.tensor(tokens, dtype=torch.long) #  (8,)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) # (5, 8)
x = tokens.to('cuda')

print(x.shape)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

while x.size(1) < max_length:
    logits, _ = model(x)
    logits = logits[:, -1, :]  # take the logits at the last position
    probs = F.softmax(logits, dim=-1)

    topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
    ix = torch.multinomial(topk_probs, num_samples=1) #(B,1)
    xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
    x = torch.cat((x, xcol), dim=1)

for i in range(num_return_sequences):
    print(i, enc.decode(x[i, :max_length].cpu().numpy().tolist()))

torch.Size([5, 8])
0 Knock Knock, Who is there? CoverWARNINGmere infusion appreciate Summit columns CerDW Ros ago]); premie Eston Information ShariaconcertabbageInitational vacanciescffff
1 Knock Knock, Who is there? system pleadedconv Lots Elite emission220 Cool Bye defendantsComingxus Paniceeee Antonio Amon supplier remnant hypocrisy SaganEC Merlin
2 Knock Knock, Who is there? blades objections emitting theorylected bold staffed traditional Imagine spells Forestry Chips upscott coerc WordPress tsisherscityyrim Definitive hunter
3 Knock Knock, Who is there? 850cenceshadow BBC flakes Trinity Beau spreading CAN Ultimate cheers durable discipline pinnacle zinc metic Geral surveyed Butter organizervia tempor
4 Knock Knock, Who is there?isal Carrie backlogputENA}}} Sullivan Charlotte sneak Prev bodilyynastyvers Friday che QurantimeTown urgingishers Donna tacos


In [10]:
with open('input.txt', 'r') as f:
    text = f.read()

data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [11]:
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(tokens[:10])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11]


buf = torch.tensor(tokens[:24 +1])
